In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import model to Console for AITRIOS

This notebook explains the workflow for importing model to "Console".

Instructions are described in [README.md](./README.md).

**ATTENTION!** <br>
The Console client used in this notebook is **not ConsoleAccessLibrary**, but a proprietary Client generated using OpenAPI Generator.

## Import modules

In [ ]:
import errno
import json
from pathlib import Path

import jsonschema
import yaml

from console_access_api.aitrios_console import AitriosConsole

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )

configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

# Set default network_type
if json_load.get("network_type") is None:
    json_load.update({"network_type": "0"})

## Initialize Console access client

In [ ]:
console_setting_path = Path("./console_access_settings.yaml")
validate_symlink(console_setting_path)

with open(console_setting_path, "r") as f:
    yaml_load = yaml.safe_load(f)
    BASE_URL = yaml_load["console_access_settings"]["console_endpoint"]
    CLIENT_ID = yaml_load["console_access_settings"]["client_id"]
    CLIENT_SECRET = yaml_load["console_access_settings"]["client_secret"]
    GCS_OKTA_DOMAIN = yaml_load["console_access_settings"]["portal_authorization_endpoint"]

console_access_api = AitriosConsole(BASE_URL, CLIENT_ID, CLIENT_SECRET, GCS_OKTA_DOMAIN)

## Upload AI Model

Place the AI model (.tflite format) in the same directory. 

When the code block is executed, an input form will appear, in which you enter the file name of the model (if nothing is entered, the default value *posenet_mobilenet_v1_075_353_481_quant.tflite* will be used).

In [ ]:
# get model name
default_file_name = "posenet_mobilenet_v1_075_353_481_quant.tflite"

input_file_name = input()
if input_file_name != None and input_file_name != "":
    file_name = input_file_name
else:
    file_name = default_file_name

file_path = Path(f"./{file_name}")
validate_symlink(file_path)

# open model file
with open(file_path, "rb") as f:
    # Call an API to upload model file
    try:
        response = console_access_api.UploadFile(type_code="NonConverted", file_name=file_name, file=f)
    except Exception as e:
        raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
model_file_id = response["file_info"]["id"]
model_name = response["file_info"]["name"]

print("Model file uploaded successfully." + " \n\tmodel_name: " + model_name + " \n\tmodel_file_id: " + model_file_id)

# add assigned id to configuration
json_load.update({"model_file_id": model_file_id})

## Import AI model

In [ ]:
# Call an API to import AI model into Console for AITRIOS
try:
    response = console_access_api.ImportBaseModel(json_load)
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Start to import the AI model." + " \n\tmodel_id: " + json_load["model_id"])

## Get AI model status

In [ ]:
def get_base_model_status(model_id: str):
    # AI model status on Console
    status_dictionary = {
        "00": "Conversion project created",
        "01": "Importing completed (Before conversion)",
        "02": "Converting...",
        "03": "Conversion failed",
        "04": "Converted",
        "05": "Adding to configuration",
        "06": "Conversion failed",
        "07": "Conversion completed",
        "11": "Saving",
    }
    # Flag for check
    exist_flag = False
    # Call an API for get AI model info
    try:
        response = console_access_api.GetBaseModelStatus(model_id=model_id)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    # Create output list
    if "projects" in response:
        project = response["projects"][0]
        if "versions" in project:
            version_status = project["versions"][0]["version_status"]
            exist_flag = True

    if exist_flag:
        return status_dictionary.get(
            version_status, "Unknown status '" + version_status + "'"
        )
    else:
        raise Exception("AI model is not found. (model_id: " + model_id + ")")


get_status = get_base_model_status(json_load["model_id"])
print(get_status + " \n\tmodel_id: " + json_load["model_id"])

## Convert AI model

In [ ]:
# Call an API to convert AI model on Console for AITRIOS
try:
    response = console_access_api.PublishModel(model_id=json_load["model_id"])
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Start to convert the AI model." + " \n\tmodel_id: " + json_load["model_id"])

## Get AI model status after conversion

Converting takes time.

To complete the conversion, ensure that the conversion status is Conversion completed.

After you start the conversion, run the following code cell to check the status :

In [ ]:
convert_status = get_base_model_status(json_load["model_id"])
print(convert_status + " \n\tmodel_id: " + json_load["model_id"])